# Genetic Toggle Switch
***
Gardner et al. Nature (1999) 'Construction of a genetic toggle switch in Escherichia coli'
***
## Setup the Environment
***

In [ ]:
import numpy
from tsfresh.feature_extraction.settings import MinimalFCParameters
from dask.distributed import Client
from sklearn.metrics import mean_absolute_error

In [ ]:
import gillespy2

In [ ]:
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.distancefunctions import naive_squared
from sciope.inference.abc_inference import ABC

***
## Create the Genetic Toggle Switch Model
***

In [ ]:
def create_genetic_toggle_switch(parameter_values=None):
    model = gillespy2.Model(name="Toggle_Switch")
    model.volume = 1

    # Variables
    U = gillespy2.Species(name="U", initial_value=10, mode="discrete")
    V = gillespy2.Species(name="V", initial_value=10, mode="discrete")
    model.add_species([U, V])

    # Parameters
    alpha1 = gillespy2.Parameter(name="alpha1", expression="1")
    alpha2 = gillespy2.Parameter(name="alpha2", expression="1")
    beta = gillespy2.Parameter(name="beta", expression="2")
    gamma = gillespy2.Parameter(name="gamma", expression="2")
    mu = gillespy2.Parameter(name="mu", expression="1")
    model.add_parameter([alpha1, alpha2, beta, gamma, mu])

    # Reactions
    cu = gillespy2.Reaction(
        name="cu",
        reactants={}, products={'U': 1},
        propensity_function="alpha1/(1+pow(V,beta))",
        ode_propensity_function="alpha1/(1+pow(V,beta))"
    )
    cv = gillespy2.Reaction(
        name="cv",
        reactants={}, products={'V': 1},
        propensity_function="alpha2/(1+pow(U,gamma))",
        ode_propensity_function="alpha2/(1+pow(U,gamma))"
    )
    du = gillespy2.Reaction(
        name="du", rate="mu",
        reactants={'U': 1}, products={}
    )
    dv = gillespy2.Reaction(
        name="dv", rate="mu",
        reactants={'V': 1}, products={}
    )
    model.add_reaction([cu, cv, du, dv])

    # Timespan
    tspan = gillespy2.TimeSpan.arange(1, t=101)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_genetic_toggle_switch()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    solver = gillespy2.SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

***
## Model Inference
***
### Generate some fixed(observed) data based on default parameters of the model

In [ ]:
kwargs = configure_simulation()
fixed_data = model.run(**kwargs)

Reshape the data and remove timepoints array

In [ ]:
fixed_data = fixed_data.to_array()
fixed_data = numpy.asarray([x.T for x in fixed_data])
fixed_data = fixed_data[:, 1:, :]

### Define prior distribution
Take default from mode 1 as reference

In [ ]:
default_param = numpy.array(list(model.listOfParameters.items()))[:, 1]

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = numpy.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

### Define simulator

In [ ]:
def get_variables(params, model):
    # params - array, need to have the same order as model.listOfParameters
    variables = {}
    for e, pname in enumerate(model.listOfParameters.keys()):
        variables[pname] = params[e]
    return variables

Here we use the GillesPy2 Solver

In [ ]:
def simulator(params, model):
    variables = get_variables(params, model)

    res = model.run(**kwargs, variables=variables)
    res = res.to_array()
    tot_res = numpy.asarray([x.T for x in res]) # reshape to (N, S, T)
    # should not contain timepoints
    tot_res = tot_res[:, 1:, :]

    return tot_res

Wrapper, simulator function to abc should should only take one argument (the parameter point)

In [ ]:
def simulator2(x):
    return simulator(x, model=model)

### Define summary statistics and distance function
Function to generate summary statistics

In [ ]:
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

### Start local cluster using dask client

In [ ]:
c = Client()
c

***
## Run the abc instance
***

In [ ]:
abc = ABC(
    fixed_data, sim=simulator2, prior_function=uni_prior,
    summaries_function=summ_func.compute, distance_function=ns
)

First compute the fixed(observed) mean

In [ ]:
abc.compute_fixed_mean(chunk_size=2)

In [ ]:
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)

In [ ]:
mae_inference = mean_absolute_error(bound, abc.results['inferred_parameters'])